#Verify the various relationships between pupil magnification and direction cosines

##References

<div class="cite2c-biblio"></div>

In [1]:
# Common imports for both sections
from __future__ import print_function, division
import numpy as np
import os
import pyzdde.zdde as pyz
import iutils.math.general as mgu
import iutils.optics.fourier as fou

##TOC

1. [Relation between pupil magnification and direction cosines](#Relation-between-pupil-magnification-and-direction-cosines)

2. [Relation between direction cosines at the ENP and the EXP](#Relation-between-direction-cosines-at-the-ENP-and-the-EXP)



##Relation between pupil magnification and direction cosines

###Introduction

One of the definitions of the **pupil magnification**, $m_p$, is the ratio of the exit pupil diameter to the entrance pupil diameter:

$$
m_p = \frac{EXP_{diameter}}{ENP_{diameter}} \hspace{40pt} (1)
$$

The pupil $m_p$ also relates the input angle ($\theta$) at the $ENP$ and the output angle ($\theta'$) at the $EXP$ that a chief ray makes with the optical axis <cite data-cite="443206/X2FEQBXG"></cite>:

$$
m_p = \frac{tan(\theta)}{tan(\theta')} \hspace{40pt}(2)
$$

Using the above relation, we can derive a relation between the input (to the $ENP$) and output (from the $EXP$) direction cosines associated with the chief ray.

####Derivation of the relations

**<font color='red'>TO DO</font>**

[Import from handwritten derivation]

###Verification using Zemax

In [2]:
ln = pyz.createLink()
lens = "paraxialDoubleLensF180mm_EnppExpp36_88mm_pMag0_82_TiltAbtENPP_wSensorShift.zmx"
zmxdir = os.path.join(os.getcwd(), "zmxfiles")
lensPath = os.path.join(zmxdir, lens)
ln.zLoadFile(lensPath)

0

In [3]:
ln.zSetConfig(config=1)

(1, 3, 0)

In [4]:
mp = ln.zGetPupilMagnification()
mp

0.8225806452000001

In order to get the direction cosines at the two surfaces the chief ray is traced in Zemax and the direction cosines at the relevant surfaces are querried.


In [13]:
# Helper function 

def get_dircos_pupilmag(hx=0, hy=1):
    '''returns direction cosines of the chief-ray at the ENP and EXP
       and the pupil magnifications computed using the 3 relations
       
    Parameters
    ----------
    hx, hy: floats
        normalized field position of the ray in the object space
    
    Returns
    -------
    a, b, g : floats
        direction cosines alpha, beta & gamma @ ENP
    aDash, bDash, gDash : floats
        direction cosines alpha, beta & gamma @ EXP
    mp1, mp2, mp3 : floats
        pupil magnification using the 3 relations
    '''
    # trace ray to the Entrance Pupil Surface (surface 9)
    rt1 = ln.zGetTrace(waveNum=1, mode=0, surf=9, hx=hx, hy=hy, px=0, py=0)
    a, b, g = rt1.dcos_l, rt1.dcos_m, rt1.dcos_n
    # trace ray to the surface just after the Exit Pupil surface (EXP surface = 14)
    rt2 = ln.zGetTrace(waveNum=1, mode=0, surf=15, hx=hx, hy=hy, px=0, py=0)
    aDash, bDash, gDash = rt2.dcos_l, rt2.dcos_m, rt2.dcos_n
    # compute the pupil magnification using the three relationships
    with np.errstate(divide='ignore', invalid='ignore'):
        mp1 = np.divide(a, aDash)*np.divide(gDash, g)
        mp2 = np.divide(b, bDash)*np.divide(gDash, g)
        mp3 = np.sqrt((a**2 + b**2)/(aDash**2 + bDash**2))*np.divide(gDash, g)
    return a, b, g, aDash, bDash, gDash, mp1, mp2, mp3 
    

def print_dircos_pupilmag(data, ppm=True, verbose=True):
    '''print the direction cosines and pupil magnification
    
    Parameters
    ----------
    data : tuple
        the tuple returned by the function get_dircos_pupilmag()
    ppm : bool
        if `True`, the pupil magnifications are printed
    verbose : bool
        if `True`, the polar, azimuth angles and angles w.r.t. the
        axes.
    
    Returns
    -------
    None
    '''
    a, b, g, aDash, bDash, gDash, mp1, mp2, mp3 = data
    zen, azi = fou.zenith_azimuth_from_dir_cos(g, a, b)
    zenDash, aziDash = fou.zenith_azimuth_from_dir_cos(gDash, aDash, bDash)
    angZ, angX, angY = fou.angles_from_dir_cos(g, a, b)
    angZDash, angXDash, angYDash = fou.angles_from_dir_cos(gDash, aDash, bDash)
    print('Direction cosines @ ENP:')
    print('alpha =', a, ', beta =', b, ', gamma =', g)
    if verbose:
        print('zenith =', zen, ', azimuth =', azi)
        print('angle_x =', angX, ', angle_y =', angY, ', angle_z =', angZ)
    print('\nDirection cosines @ EXP:')
    print("alpha' =", aDash, ", beta' =", bDash, ", gamma' =", gDash)
    if verbose:
        print("zenith' =", zenDash, ", azimuth' =", aziDash)
        print("angle_x' =", angXDash, ", angle_y' =", angYDash, ", angle_z' =", angZDash)
    if ppm:
        print('\nPupil Magnification:')
        print('mp1 (def#1) =', mp1, ', mp2 (def#2) =', mp2, ', mp3 (def#3) =', mp3)   


######Trace a chief-ray that is in the Tangential plane

In [14]:
ret = get_dircos_pupilmag(hx=0, hy=1)
print_dircos_pupilmag(ret)

Direction cosines @ ENP:
alpha = 0.0 , beta = -0.18721331631 , gamma = 0.982319283225
zenith = 10.7902009975 , azimuth = -90.0
angle_x = 90.0 , angle_y = 100.790200998 , angle_z = 10.7902009975

Direction cosines @ EXP:
alpha' = 0.0 , beta' = -0.225710231566 , gamma' = 0.974194483338
zenith' = 13.0446458379 , azimuth' = -90.0
angle_x' = 90.0 , angle_y' = 103.044645838 , angle_z' = 13.0446458379

Pupil Magnification:
mp1 (def#1) = nan , mp2 (def#2) = 0.822580645161 , mp3 (def#3) = 0.822580645161


######Trace a chief ray that is in the sagittal plane

In [17]:
ret = get_dircos_pupilmag(hx=1, hy=0)
print_dircos_pupilmag(ret)

Direction cosines @ ENP:
alpha = -0.18721331631 , beta = 0.0 , gamma = 0.982319283225
zenith = 10.7902009975 , azimuth = 180.0
angle_x = 100.790200998 , angle_y = 90.0 , angle_z = 10.7902009975

Direction cosines @ EXP:
alpha' = -0.225710231566 , beta' = 0.0 , gamma' = 0.974194483338
zenith' = 13.0446458379 , azimuth' = 180.0
angle_x' = 103.044645838 , angle_y' = 90.0 , angle_z' = 13.0446458379

Pupil Magnification:
mp1 (def#1) = 0.822580645161 , mp2 (def#2) = nan , mp3 (def#3) = 0.822580645161


######Trace a ray that is neither in the tangential plane nor in the sagittal plane

In [18]:
ret = get_dircos_pupilmag(hx=0.707, hy=0.707)
print_dircos_pupilmag(ret)

Direction cosines @ ENP:
alpha = -0.132360515134 , beta = -0.132360515134 , gamma = 0.982324482066
zenith = 10.7886098001 , azimuth = -135.0
angle_x = 97.60601883 , angle_y = 97.60601883 , angle_z = 10.7886098001

Direction cosines @ EXP:
alpha' = -0.159578361312 , beta' = -0.159578361312 , gamma' = 0.974201977622
zenith' = 13.0427433031 , azimuth' = -135.0
angle_x' = 99.1824236599 , angle_y' = 99.1824236599 , angle_z' = 13.0427433031

Pupil Magnification:
mp1 (def#1) = 0.822580645161 , mp2 (def#2) = 0.822580645161 , mp3 (def#3) = 0.822580645161


In [19]:
ret = get_dircos_pupilmag(hx=0.6, hy=1.0)
print_dircos_pupilmag(ret)

Direction cosines @ ENP:
alpha = -0.111625972615 , beta = -0.186043287691 , gamma = 0.97618007424
zenith = 12.5306527065 , azimuth = -120.963756532
angle_x = 96.4090542593 , angle_y = 100.721964415 , angle_z = 12.5306527065

Direction cosines @ EXP:
alpha' = -0.134201090298 , beta' = -0.22366848383 , gamma' = 0.965382036659
zenith' = 15.1199126143 , azimuth' = -120.963756532
angle_x' = 97.7124253602 , angle_y' = 102.924592577 , angle_z' = 15.1199126143

Pupil Magnification:
mp1 (def#1) = 0.822580645161 , mp2 (def#2) = 0.822580645161 , mp3 (def#3) = 0.822580645161


In [20]:
ln.close()

##Relation between direction cosines at the ENP and the EXP

###Relationship

Let $\{\alpha, \beta, \gamma \}$ and $\{\alpha', \beta', \gamma' \}$ be the sets of direction cosines at the ENP and EXP respectively. Then the relations between the elements of the sets are given as follows:

$$
\begin{array}{cc}
\alpha' &=& \frac{1}{m_p} \left(\frac{\gamma'}{\gamma} \right) \alpha \\
\beta' &=& \frac{1}{m_p} \left(\frac{\gamma'}{\gamma} \right) \beta \\
\gamma' &=& \pm \frac{m_p}{\sqrt{1 + (m_p - 1)\gamma^2}} \gamma
\end{array}
$$

####Derivation of the relations

**<font color='red'>TO DO</font>**

[Import from handwritten derivation]

###Verification using Zemax

In [4]:
if not ln:
    ln = pyz.createLink()
lens = "paraxialDoubleLensF180mm_EnppExpp36_88mm_pMag0_82_TiltAbtENPP_wSensorShift.zmx"
zmxdir = os.path.join(os.getcwd(), "zmxfiles")
lensPath = os.path.join(zmxdir, lens)
ln.zLoadFile(lensPath)

0

In [5]:
ln.zSetConfig(config=1)

(1, 3, 0)

In [6]:
mp = ln.zGetPupilMagnification()
mp

0.8225806452000001

In [10]:
def evaluate_dircos_exp(a, b, g, mp):
    '''evaluates the direction cosines at the EXP based on the
       above expression
       
    Parameters
    ----------
    a, b, g : floats
        direction cosines, $cos(\theta_x)$, $cos(\theta_y)$, $cos(\theta_z)$,
        at the ENP
    mp : float
        pupil magnification
        
    Returns
    -------
    aDash, bDash, gDash : floats
        direction cosines at the EXP
    
    Note
    ----
    The function evaluates and returns only the positive value of gDash.
    '''
    gDash = mp*g/np.sqrt(1.0 + (mp**2 - 1)*g**2)
    comnFactor = (1.0/mp)*(gDash/g)
    aDash = comnFactor*a
    bDash = comnFactor*b
    return aDash, bDash, gDash

######Trace a chief-ray that is in the Tangential plane

In [15]:
ret = get_dircos_pupilmag(hx=0, hy=1)
print_dircos_pupilmag(ret, False, False)
aDash, bDash, gDash = evaluate_dircos_exp(a=ret[0], b=ret[1], g=ret[2], mp=ret[-1])
print("\nDirection cosine @ EXP (EVALUATED):")
print("alpha' =", aDash, ", beta' =", bDash, ", gamma' =", gDash)

Direction cosines @ ENP:
alpha = 0.0 , beta = -0.18721331631 , gamma = 0.982319283225

Direction cosines @ EXP:
alpha' = 0.0 , beta' = -0.225710231566 , gamma' = 0.974194483338

Direction cosine @ EXP (EVALUATED):
alpha' = 0.0 , beta' = -0.225710231566 , gamma' = 0.974194483338


######Trace a chief ray that is in the sagittal plane

In [17]:
ret = get_dircos_pupilmag(hx=1, hy=0)
print_dircos_pupilmag(ret, False, False)
aDash, bDash, gDash = evaluate_dircos_exp(a=ret[0], b=ret[1], g=ret[2], mp=ret[-1])
print("\nDirection cosine @ EXP (EVALUATED):")
print("alpha' =", aDash, ", beta' =", bDash, ", gamma' =", gDash)

Direction cosines @ ENP:
alpha = -0.18721331631 , beta = 0.0 , gamma = 0.982319283225

Direction cosines @ EXP:
alpha' = -0.225710231566 , beta' = 0.0 , gamma' = 0.974194483338

Direction cosine @ EXP (EVALUATED):
alpha' = -0.225710231566 , beta' = 0.0 , gamma' = 0.974194483338


######Trace a ray that is neither in the tangential plane nor in the sagittal plane

In [18]:
ret = get_dircos_pupilmag(hx=0.6, hy=1.0)
print_dircos_pupilmag(ret, False, False)
aDash, bDash, gDash = evaluate_dircos_exp(a=ret[0], b=ret[1], g=ret[2], mp=ret[-1])
print("\nDirection cosine @ EXP (EVALUATED):")
print("alpha' =", aDash, ", beta' =", bDash, ", gamma' =", gDash)

Direction cosines @ ENP:
alpha = -0.111625972615 , beta = -0.186043287691 , gamma = 0.97618007424

Direction cosines @ EXP:
alpha' = -0.134201090298 , beta' = -0.22366848383 , gamma' = 0.965382036659

Direction cosine @ EXP (EVALUATED):
alpha' = -0.134201090298 , beta' = -0.22366848383 , gamma' = 0.965382036659


In [19]:
ln.close()